<a href="https://colab.research.google.com/github/wurDevTim/Workshop_P4P/blob/main/correcting_meassurement_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Correcting for meassurement time
workshop part 2:


## setup
Next to python, R code can also be used in colab notebooks.
Here we call R from python instead with rpy2 to enable us to use python code as well.
An R cell is marked with '%%R' at the top of the cell.

In [4]:
%load_ext rpy2.ipython

In [62]:
# indicate that you're running R code
%%R

# Install packages
install.packages("LMMsolver")


(as ‘lib’ is unspecified)







	‘/tmp/RtmpEbqQZs/downloaded_packages’



In [63]:
# Import
%%R
library(LMMsolver)

In [7]:
# Mount google drive - not found an R alternative, using python instead.
from google.colab import drive
from os import path

datafolder = "/content/drive/My Drive/P4P_workshop_data"
# Check if the data folder is mounted correctly
if not path.exists(datafolder):
  drive.mount('/content/drive')

!ls "$datafolder"

Mounted at /content/drive
 cropreporter_traits.csv  'Enza Mandy Boon'   Lucia   npec_tomato


In [112]:
# Function which uses the LMM solver to compute the spline.
%%R

compute_spline <- function(df, plant_identifier, trait_list)
{
  ### Fit 1D spline per plant
  for (i in c(1:length(unique(df[[plant_identifier]])))){
    plant_id = unique(df[[plant_identifier]])[i]
    one_plant <- df[df[[plant_identifier]] == plant_id,]
    datenum = one_plant[['datenum']]
    preddates <- data.frame(datenum = min(one_plant$date):max(one_plant$date))
    # Each day has 24*60*60 = 86400 hours
    preddates <- preddates * 86400

    # Fit 1D spline per trait
    for (trait in trait_list){
      # Check for inf values
      if (sum(is.infinite(one_plant[[trait]])) > 0) {
        print(paste('Warning: infinite value encoutered for plant: ', plant_id, ', trait: ',trait))
      }
      trait_df <- one_plant[!is.infinite(one_plant[[trait]]),]
      # Need at least 2 not inf values
      # Need at least 2 not inf values
      if (nrow(trait_df) > 2) {
        # Nan values will be removed, but they do cause warnings.
        m1 <- LMMsolve(fixed = as.formula(paste(trait, "~", 1)),
                       spline = ~spl1D(x = datenum, nseg = 20),
                       data = trait_df)
        summary(m1)
        prediction <- obtainSmoothTrend(m1, newdata = preddates,
                                        includeIntercept = T)
        # Rename ypred column
        names(prediction)[names(prediction) == 'ypred'] <- trait
      } else {
        print(paste('Warning: not enough values to process plant: ', plant_id, ', trait: ',trait))
        prediction <- preddates
        prediction[trait] <- NA
      }
      prediction <- prediction[,c("datenum",trait)]
      # Combine results
      if (trait == trait_list[1]){
        plant_predictions <- prediction
      } else {
        plant_predictions <- merge(plant_predictions, prediction, by='datenum')
      }
    }
    plant_predictions[[plant_identifier]] = plant_id
    if (i == 1){
      all_predictions <- plant_predictions
    } else {
      all_predictions <- rbind(all_predictions, plant_predictions)
    }
  }
  return(all_predictions)
}

# Example
In this example the cropreporter data from Lucia is used, which has been analysed beforehand.

Note: Systems like the cropreporter in NPEC use the local time. If your unlucky your experiment contains both winter & summer time. In this case we would advice to switch to UTC.

In [80]:
# Load the data
%%R
df <- read.csv('/content/drive/My Drive/P4P_workshop_data/cropreporter_traits.csv', sep = ",")
head(df)

                                                                                                                                                                                                               filename
1     D:/temp/p4p/Lucia\\PMD_PML_(Control_vs_Drought_one_genotype)\\PMD_protocol\\20230708\\NPEC52.20230605.BD22.CE3027.Control.2\\CropReporter\\163111171\\data\\HDR_90_NPEC52.20230605.BD22.CE3027.Control.2_1368.INF
2       D:/temp/p4p/Lucia\\PMD_PML_(Control_vs_Drought_one_genotype)\\PMD_protocol\\20230708\\NPEC52.20230605.BD4.CE3027.Control.1\\CropReporter\\124326957\\data\\HDR_90_NPEC52.20230605.BD4.CE3027.Control.1_1368.INF
3     D:/temp/p4p/Lucia\\PMD_PML_(Control_vs_Drought_one_genotype)\\PMD_protocol\\20230708\\NPEC52.20230605.BE11.CE3027.Control.3\\CropReporter\\142623408\\data\\HDR_90_NPEC52.20230605.BE11.CE3027.Control.3_1368.INF
4     D:/temp/p4p/Lucia\\PMD_PML_(Control_vs_Drought_one_genotype)\\PMD_protocol\\20230708\\NPEC52.20230605.BE29.CE3027.Control.4\\CropR

In [81]:
%%R
# Converting to datetime object
df[['Datetime']] <- as.POSIXct(df[['Datetime']], format = "%Y-%m-%d, %H:%M:%OS")

# Datenum stored as integer, exact datetime of measurement
df[['datenum']] <- as.integer(df[['Datetime']])

# Date is multiplied with 86400 to get value at 00:00:00 of each day
df[['date']] <- as.numeric(as.Date(df[['Datetime']]))

In [82]:
%%R
head(df)

                                                                                                                                                                                                               filename
1     D:/temp/p4p/Lucia\\PMD_PML_(Control_vs_Drought_one_genotype)\\PMD_protocol\\20230708\\NPEC52.20230605.BD22.CE3027.Control.2\\CropReporter\\163111171\\data\\HDR_90_NPEC52.20230605.BD22.CE3027.Control.2_1368.INF
2       D:/temp/p4p/Lucia\\PMD_PML_(Control_vs_Drought_one_genotype)\\PMD_protocol\\20230708\\NPEC52.20230605.BD4.CE3027.Control.1\\CropReporter\\124326957\\data\\HDR_90_NPEC52.20230605.BD4.CE3027.Control.1_1368.INF
3     D:/temp/p4p/Lucia\\PMD_PML_(Control_vs_Drought_one_genotype)\\PMD_protocol\\20230708\\NPEC52.20230605.BE11.CE3027.Control.3\\CropReporter\\142623408\\data\\HDR_90_NPEC52.20230605.BE11.CE3027.Control.3_1368.INF
4     D:/temp/p4p/Lucia\\PMD_PML_(Control_vs_Drought_one_genotype)\\PMD_protocol\\20230708\\NPEC52.20230605.BE29.CE3027.Control.4\\CropR

In [83]:
%%R
# All columns
colnames(df)

 [1] "filename"        "Datetime"        "ExperimentId"    "TreatmentId"    
 [5] "PlantId"         "CameraHeight"    "ConfigFile"      "MeanNdvi"       
 [9] "MeanEgreen"      "MeanPsri"        "MeanAri"         "MeanMari"       
[13] "MeanChlorophyll" "min_yii"         "mean_yii"        "max_yii"        
[17] "datenum"         "date"           


In [114]:
%%R
# List of the columns to interpolate
trait_list <- list('mean_yii', 'MeanChlorophyll', 'MeanNdvi', 'MeanEgreen', 'MeanPsri', 'MeanAri', 'MeanMari')


In [113]:
%%R
# Compute the spline
predictions = compute_spline(df, 'PlantId', trait_list)

[1] "Warning: infinite value encoutered for plant:  NPEC52.20230605.BD22.CE3027.Control.2 , trait:  mean_yii"
[1] "Warning: infinite value encoutered for plant:  NPEC52.20230605.BD4.CE3027.Control.1 , trait:  mean_yii"
[1] "Warning: infinite value encoutered for plant:  NPEC52.20230605.BE11.CE3027.Control.3 , trait:  mean_yii"
[1] "Warning: infinite value encoutered for plant:  NPEC52.20230605.BE29.CE3027.Control.4 , trait:  mean_yii"
[1] "Warning: infinite value encoutered for plant:  NPEC52.20230605.BF18.CE3027.Control.5 , trait:  mean_yii"
[1] "Warning: infinite value encoutered for plant:  NPEC52.20230605.BG25.CE3027.Drought_2.7 , trait:  mean_yii"
[1] "Warning: infinite value encoutered for plant:  NPEC52.20230605.BH14.CE3027.Drought_2.8 , trait:  mean_yii"
[1] "Warning: not enough values to process plant:  NPEC52.20230605.BH14.CE3027.Drought_2.8 , trait:  mean_yii"
[1] "Warning: infinite value encoutered for plant:  NPEC52.20230605.BH32.CE3027.Drought_2.9 , trait:  mean_yii"
[1] 

In [117]:
%%R
# Convert time back to datetime
predictions[['Datetime']] <- as.POSIXct(predictions[['datenum']]	, origin="1970-01-01")

In [118]:
%%R
# Look at some results
head(predictions[predictions[['PlantId']] == unique(predictions[['PlantId']])[1],])

     datenum mean_yii MeanChlorophyll  MeanNdvi MeanEgreen  MeanPsri  MeanAri
1 1686787200    0.725        16919.50 0.7979598  0.5189260 0.3893428 12.89457
2 1686873600    0.725        17103.15 0.8010606  0.5273796 0.3882193 12.66434
3 1686960000    0.725        17296.47 0.8044263  0.5359966 0.3870027 12.44715
4 1687046400    0.725        17489.74 0.8080578  0.5443306 0.3858316 12.23930
5 1687132800    0.725        17682.95 0.8121139  0.5522104 0.3847343 12.04657
6 1687219200    0.725        17876.07 0.8167585  0.5594559 0.3837420 11.87531
  MeanMari                               PlantId   Datetime
1 3.294881 NPEC52.20230605.BD22.CE3027.Control.2 2023-06-15
2 3.318810 NPEC52.20230605.BD22.CE3027.Control.2 2023-06-16
3 3.345530 NPEC52.20230605.BD22.CE3027.Control.2 2023-06-17
4 3.372417 NPEC52.20230605.BD22.CE3027.Control.2 2023-06-18
5 3.399569 NPEC52.20230605.BD22.CE3027.Control.2 2023-06-19
6 3.427089 NPEC52.20230605.BD22.CE3027.Control.2 2023-06-20


In [ ]:
%%R
# Save predictions
write.csv(predictions, "/content/drive/My Drive/P4P_workshop_data/pspline_predictions.csv", row.names=FALSE)

In [39]:
%%R
preddates <- data.frame(datenum = min(df$date):max(df$date))
preddates

   datenum
1    19523
2    19524
3    19525
4    19526
5    19527
6    19528
7    19529
8    19530
9    19531
10   19532
11   19533
12   19534
13   19535
14   19536
15   19537
16   19538
17   19539
18   19540
19   19541
20   19542
21   19543
22   19544
23   19545
24   19546
25   19547
26   19548
27   19549
28   19550
29   19551
30   19552
31   19553
32   19554
33   19555
34   19556
35   19557
36   19558
37   19559
38   19560
39   19561
40   19562
41   19563
